<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [11]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
!pip install -U scikit-learn scipy matplotlib
from sklearn import preprocessing
%matplotlib inline

Requirement already up-to-date: scikit-learn in /home/jupyterlab/conda/lib/python3.6/site-packages (0.20.2)
Requirement already up-to-date: scipy in /home/jupyterlab/conda/lib/python3.6/site-packages (1.2.0)
Requirement already up-to-date: matplotlib in /home/jupyterlab/conda/lib/python3.6/site-packages (3.0.2)


/home/jupyterlab/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:233: MatplotlibDeprecationWarning: The pylab_setup function was deprecated in version 3.0.
  # See https://github.com/matplotlib/matplotlib/issues/6092


AttributeError: type object 'Backend' has no attribute 'backend_version'

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


/home/jupyterlab/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:233: MatplotlibDeprecationWarning: The pylab_setup function was deprecated in version 3.0.
  # See https://github.com/matplotlib/matplotlib/issues/6092


AttributeError: type object 'Backend' has no attribute 'backend_version'

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [3]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
print('Download Successful')    

--2019-01-06 17:45:45--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[=====================>]  22.56K  --.-KB/s   in 0.02s  

2019-01-06 17:45:45 (1.05 MB/s) - ‘loan_train.csv’ saved [23101/23101]

Download Successful
Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


AttributeError: type object 'Backend' has no attribute 'backend_version'

### Load Data From CSV File  

In [4]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


AttributeError: type object 'Backend' has no attribute 'backend_version'

In [5]:
df.shape

(346, 10)

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


AttributeError: type object 'Backend' has no attribute 'backend_version'

In [6]:
df.groupby('loan_status').size()

loan_status
COLLECTION     86
PAIDOFF       260
dtype: int64

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


AttributeError: type object 'Backend' has no attribute 'backend_version'

In [8]:
!conda install -c anaconda seaborn -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libssh2-1.8.0              |       h1ba5d50_4         233 KB  anaconda
    ca-certificates-2018.03.07 |                0         124 KB  anaconda
    krb5-1.16.1                |       h173b8e3_7         1.4 MB  anaconda
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following packages will be UPDATED:

    cryptography:    2.3.1-py36hdffb7b8_0    conda-forge --> 2.4.1-py36h1ba5d50_0    anaconda
    grpcio:          1.16.0-py36hd60e7a3_0   conda-forge --> 1.16.1-py36hf8bcb03_1   anaconda
    libarchive:      3.3.3-h823be47_0        conda-forge --> 3.3.3-h5d8350f_4        anaconda
    libcurl:         7.63.0-hbdb9355_0  

/home/jupyterlab/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:233: MatplotlibDeprecationWarning: The pylab_setup function was deprecated in version 3.0.
  # See https://github.com/matplotlib/matplotlib/issues/6092


AttributeError: type object 'Backend' has no attribute 'backend_version'

In [9]:
import seaborn as sns
sns.countplot(df['loan_status'],label="Count")

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7f554e7a9268> (for post_run_cell):


/home/jupyterlab/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:233: MatplotlibDeprecationWarning: The pylab_setup function was deprecated in version 3.0.
  # See https://github.com/matplotlib/matplotlib/issues/6092


AttributeError: type object 'Backend' has no attribute 'backend_version'

### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to convert categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
#from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,jaccard_similarity_score, log_loss
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

from sklearn.neighbors import KNeighborsClassifier
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for n_neighbors in neighbors_settings:
    # build the model
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    # record training set accuracy
    training_accuracy.append(knn.score(X_train, y_train))
    # record test set accuracy
    test_accuracy.append(knn.score(X_test, y_test))
plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'.format(knn.score(X_test, y_test)))

In [ ]:
knn_predict = knn.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, knn_predict))
print('\n')
print('jaccard_similarity_score:\n\n {:.2f}' .format(jaccard_similarity_score(y_test, knn_predict)*100),'%')

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Prune the tree to account for overfitting
# Max depth of 3 levels

tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

In [ ]:
tree_predict = tree.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, tree_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, tree_predict)*100,'%')

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svc = SVC(C=0.01)
svc.fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(svc.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test, y_test)))

In [ ]:
svc_predict = svc.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, svc_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, svc_predict)*100,'%')

# Logistic Regression

In [ ]:
from sklearn.metrics import classification_report,jaccard_similarity_score, log_loss

# Using C = 0.01 to improve better scoring

logreg = LogisticRegression(C=0.01).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test, y_test)))

In [ ]:
log_reg_predict = logreg.predict(X_test)
log_reg_predict_proba = logreg.predict_proba(X_test)[:, 1]
print('Classification report:\n\n', classification_report(y_test, log_reg_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, log_reg_predict)*100,'%')


# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)
#df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
X = Feature
y = df['loan_status'].values

# Train-Test-Split

In [ ]:
#Test_Train_Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

# K Nearest Neighbor(KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for n_neighbors in neighbors_settings:
    # build the model
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    # record training set accuracy
    training_accuracy.append(knn.score(X_train, y_train))
    # record test set accuracy
    test_accuracy.append(knn.score(X_test, y_test))
plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'.format(knn.score(X_test, y_test)))

In [ ]:
knn_predict = knn.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, knn_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, knn_predict)*100,'%')

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Prune the tree to account for overfitting
# Max depth of 3 levels

tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

In [ ]:
tree_predict = tree.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, tree_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, tree_predict)*100,'%')

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svc = SVC(C=0.01)
svc.fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(svc.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test, y_test)))

In [ ]:
svc_predict = svc.predict(X_test)
print('Classification report:\n\n', classification_report(y_test, svc_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, svc_predict)*100,'%')

# Logistic Regression

In [ ]:
from sklearn.metrics import classification_report,jaccard_similarity_score, log_loss

# Using C = 0.01 to improve better scoring

logreg = LogisticRegression(C=0.01).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test, y_test)))

In [ ]:
log_reg_predict = logreg.predict(X_test)
log_reg_predict_proba = logreg.predict_proba(X_test)[:, 1]
print('Classification report:\n\n', classification_report(y_test, log_reg_predict))
print('\n')
print('jaccard_similarity_score:\n\n', jaccard_similarity_score(y_test, log_reg_predict)*100,'%')

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>